In [1]:
import org.apache.spark.ml.feature.{StringIndexer, VectorAssembler}

import org.apache.spark.ml.classification.GBTClassifier
import org.apache.spark.ml.Pipeline
import org.apache.spark.sql.functions.{udf, col}
import org.apache.spark.ml.linalg.DenseVector

In [2]:
val spark = SparkSession.builder().appName("lab5").getOrCreate()

spark = org.apache.spark.sql.SparkSession@320ec4bf


org.apache.spark.sql.SparkSession@320ec4bf

In [3]:
val df = spark.read
    .option("delimiter", ",")
    .option("header", true)
    .option("inferSchema", "true")
    .csv("/labs/slaba05/lab05_train.csv")

df = [_c0: int, ID: int ... 115 more fields]


[_c0: int, ID: int ... 115 more fields]

In [4]:
val test = spark.read
    .option("delimiter", ",")
    .option("header", true)
    .option("inferSchema", "true")
    .csv("/labs/slaba05/lab05_train.csv")

test = [_c0: int, ID: int ... 115 more fields]


[_c0: int, ID: int ... 115 more fields]

In [5]:
val categorical_subset = List(
    "CLNT_TRUST_RELATION", 
    "APP_MARITAL_STATUS", 
    "APP_KIND_OF_PROP_HABITATION", 
    "CLNT_JOB_POSITION_TYPE",
    "APP_DRIVING_LICENSE", 
    "APP_EDUCATION", 
    "APP_TRAVEL_PASS", 
    "APP_CAR", 
    "APP_POSITION_TYPE",
    "APP_EMP_TYPE", 
    "APP_COMP_TYPE", 
    "PACK"
)

categorical_subset = List(CLNT_TRUST_RELATION, APP_MARITAL_STATUS, APP_KIND_OF_PROP_HABITATION, CLNT_JOB_POSITION_TYPE, APP_DRIVING_LICENSE, APP_EDUCATION, APP_TRAVEL_PASS, APP_CAR, APP_POSITION_TYPE, APP_EMP_TYPE, APP_COMP_TYPE, PACK)


List(CLNT_TRUST_RELATION, APP_MARITAL_STATUS, APP_KIND_OF_PROP_HABITATION, CLNT_JOB_POSITION_TYPE, APP_DRIVING_LICENSE, APP_EDUCATION, APP_TRAVEL_PASS, APP_CAR, APP_POSITION_TYPE, APP_EMP_TYPE, APP_COMP_TYPE, PACK)

In [6]:
val df_cat = df.select(categorical_subset.map(m=>col(m)):_*)

df_cat = [CLNT_TRUST_RELATION: string, APP_MARITAL_STATUS: string ... 10 more fields]


[CLNT_TRUST_RELATION: string, APP_MARITAL_STATUS: string ... 10 more fields]

In [7]:
val encoded_features = df_cat.columns.flatMap{ name => 
    val indexer = new StringIndexer()
        .setInputCol(name)
        .setOutputCol(name + "_ind")    
    Array(indexer)
}.toArray

encoded_features = Array(strIdx_abf0500ceb28, strIdx_a6cf82958584, strIdx_63dbaee2b03c, strIdx_5c03167afcb8, strIdx_ee7a9d4292c6, strIdx_cea3aee2056a, strIdx_3faa48d0e3c1, strIdx_ca5dfca33d15, strIdx_be5ee4f8590b, strIdx_c9da0f5a8815, strIdx_57b5cbebc535, strIdx_a1e5c98c228f)


Array(strIdx_abf0500ceb28, strIdx_a6cf82958584, strIdx_63dbaee2b03c, strIdx_5c03167afcb8, strIdx_ee7a9d4292c6, strIdx_cea3aee2056a, strIdx_3faa48d0e3c1, strIdx_ca5dfca33d15, strIdx_be5ee4f8590b, strIdx_c9da0f5a8815, strIdx_57b5cbebc535, strIdx_a1e5c98c228f)

In [8]:
val filled_df = df.na.fill("Unknown", categorical_subset.toArray).na.fill(0)

filled_df = [_c0: int, ID: int ... 115 more fields]


[_c0: int, ID: int ... 115 more fields]

In [9]:
val pipeline = new Pipeline().setStages(encoded_features)
val indexer_model = pipeline.fit(filled_df)

pipeline = pipeline_bda299a5495b
indexer_model = pipeline_bda299a5495b


pipeline_bda299a5495b

In [10]:
val df_transformed = indexer_model.transform(filled_df)

df_transformed = [_c0: int, ID: int ... 127 more fields]


[_c0: int, ID: int ... 127 more fields]

In [11]:
val igrnore_cols = df_cat.columns ++ Array("_c0", "ID", "TARGET", "CLNT_JOB_POSITION")
val assembler_cols = (df_transformed.columns.toSet -- igrnore_cols.toSet).toArray

igrnore_cols = Array(CLNT_TRUST_RELATION, APP_MARITAL_STATUS, APP_KIND_OF_PROP_HABITATION, CLNT_JOB_POSITION_TYPE, APP_DRIVING_LICENSE, APP_EDUCATION, APP_TRAVEL_PASS, APP_CAR, APP_POSITION_TYPE, APP_EMP_TYPE, APP_COMP_TYPE, PACK, _c0, ID, TARGET, CLNT_JOB_POSITION)
assembler_cols = Array(SUM_TRAN_CLO_TENDENCY1M, LDEAL_USED_AMT_AVG_YWZ, CNT_TRAN_AUT_TENDENCY3M, AMOUNT_RUB_SUP_PRC, REST_DYNAMIC_PAYM_3M, LDEAL_ACT_DAYS_PCT_TR3, PRC_ACCEPTS_A_MTP, AMOUNT_RUB_CLO_PRC, APP_KIND_OF_PROP_HABITATION_ind, TURNOVER_DYNAMIC_PAYM_3M, APP_DRIVING_LICENSE_ind, AMOUNT_RUB_NAS_PRC, CR_PROD_CNT_CCFP, LDEAL_TENOR_MAX, CNT_TRAN_SUP_TENDENCY3M, SUM_TRAN_AUT_TENDENCY3M, CNT_TRAN_MED_TENDENCY3M, REST_AVG_PAYM, REST_DYNAMIC_CC_3M, DEAL_YWZ_IR_MAX, PRC_ACCEPTS_A_ATM, MAX_PCLOSE_DA...


Array(SUM_TRAN_CLO_TENDENCY1M, LDEAL_USED_AMT_AVG_YWZ, CNT_TRAN_AUT_TENDENCY3M, AMOUNT_RUB_SUP_PRC, REST_DYNAMIC_PAYM_3M, LDEAL_ACT_DAYS_PCT_TR3, PRC_ACCEPTS_A_MTP, AMOUNT_RUB_CLO_PRC, APP_KIND_OF_PROP_HABITATION_ind, TURNOVER_DYNAMIC_PAYM_3M, APP_DRIVING_LICENSE_ind, AMOUNT_RUB_NAS_PRC, CR_PROD_CNT_CCFP, LDEAL_TENOR_MAX, CNT_TRAN_SUP_TENDENCY3M, SUM_TRAN_AUT_TENDENCY3M, CNT_TRAN_MED_TENDENCY3M, REST_AVG_PAYM, REST_DYNAMIC_CC_3M, DEAL_YWZ_IR_MAX, PRC_ACCEPTS_A_ATM, MAX_PCLOSE_DA...

In [12]:
val assembler = new VectorAssembler()
                    .setInputCols(assembler_cols.map(_.toString))
                    .setOutputCol("features")

assembler = vecAssembler_feda7e7b10a2


vecAssembler_feda7e7b10a2

In [13]:
val assembler = new VectorAssembler()
                    .setInputCols(assembler_cols.map(_.toString))
                    .setOutputCol("features")

val gbt = new GBTClassifier().setFeaturesCol("features").setLabelCol("TARGET")

val pipelineGBT = new Pipeline().setStages(Array(assembler, gbt))

assembler = vecAssembler_6b428790bdba
gbt = gbtc_cc4da1ce86a4
pipelineGBT = pipeline_e5d945af7216


pipeline_e5d945af7216

In [14]:
val model = pipelineGBT.fit(df_transformed)

model = pipeline_e5d945af7216


pipeline_e5d945af7216

In [15]:
val filled_test = test.na.fill("Unknown", categorical_subset.toArray).na.fill(0)

filled_test = [_c0: int, ID: int ... 115 more fields]


[_c0: int, ID: int ... 115 more fields]

In [16]:
val df_test = indexer_model.transform(filled_test)

df_test = [_c0: int, ID: int ... 127 more fields]


[_c0: int, ID: int ... 127 more fields]

In [17]:
val prediction = model.transform(df_test)

prediction = [_c0: int, ID: int ... 131 more fields]


[_c0: int, ID: int ... 131 more fields]

In [18]:
val toArr: Any => Array[Double] = _.asInstanceOf[DenseVector].toArray
val toArrUdf = udf(toArr)

toArr = > Array[Double] = <function1>
toArrUdf = UserDefinedFunction(<function1>,ArrayType(DoubleType,false),None)


UserDefinedFunction(<function1>,ArrayType(DoubleType,false),None)

In [19]:
val result = prediction.select(col("ID").alias("id"), toArrUdf(col("probability")).getItem(1).alias("target"))

result = [id: int, target: double]


[id: int, target: double]

In [20]:
result.coalesce(1).write
    .option("header", "true")
    .option("delimiter","\t")
    .mode("overwrite")
    .csv("lab05.csv")